In [1]:
import pickle
import re
from tqdm import tqdm
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from scipy.spatial import distance
from heapq import nsmallest
from sklearn.metrics import pairwise_distances

In the block below we are reading the raw data from articles and tables.

In [2]:
rawArticles = pickle.load(open("NewsDataset/final_article_tables.pkl","rb"))
rawTables = pickle.load(open("NewsDataset/final_dict_tables.pkl","rb"))

In the block next, we are formatting the articles data by removing links, multiples spaces and special characters. Each article has several links. Then we get each one. Finally, we read all text of each link.

In [3]:
formattedArticles = []

for i in tqdm(rawArticles.keys()):
    
    keyArticle = i
     
    dataArticle = rawArticles[keyArticle]

    totalArticles = len(dataArticle)

    for i in range(totalArticles):
    
        fullText = str(dataArticle[i][1])
    
        #removing special characters
        formattedText = re.sub('[^A-Za-z]+',' ',fullText)
    
        #removing left and right spaces
        formattedText = formattedText.lstrip()
        formattedText = formattedText.rstrip()
    
        formattedArticles.append([keyArticle,formattedText])

100%|██████████| 90000/90000 [02:44<00:00, 548.52it/s]


In the next block, we remove empty values from articles.

In [4]:
for article in tqdm(formattedArticles):
    if (article[1] == ''):
        formattedArticles.remove(article)

 98%|█████████▊| 291676/298725 [00:47<00:01, 6117.12it/s]


In the block next, we are reading and formatting the raw table data. Each indice may have multiple tables. So, we collect each table separately. Next, we add tables in the corpus.

In [43]:
formattedTables = []

for i in tqdm(rawTables.keys()):
    
    dataTable = rawTables[i]
    
    totalTalbes = len(dataTable)
    
    for j in range(totalTalbes):
        
        keyTable = dataTable[j][0][0]
        
        keyCaption = dataTable[j][0][1]
        
        tableText = str(dataTable[j][1:])
    
        fullTextTable = re.sub('[^A-Za-z]+',' ',tableText)
        
        #removing left and right spaces
        fullTextTable = fullTextTable.lstrip()
        fullTextTable = fullTextTable.rstrip()
        
        textPlusCaption = fullTextTable +" "+keyCaption
        
        formattedTables.append([keyTable,keyCaption,fullTextTable,textPlusCaption])


 51%|█████     | 45618/90000 [00:17<00:18, 2340.69it/s]


 96%|█████████▌| 86000/90000 [00:34<00:01, 2128.99it/s]


100%|██████████| 90000/90000 [00:36<00:00, 2481.76it/s]

In [44]:
for tables in tqdm(formattedTables):
    if (tables[2] == ''):
        formattedTables.remove(tables)


 41%|████      | 126825/308759 [00:16<00:42, 4254.46it/s]


 63%|██████▎   | 193406/308759 [00:33<00:36, 3201.80it/s]


 82%|████████▏ | 254626/308759 [00:52<00:25, 2091.55it/s]


 97%|█████████▋| 298855/308759 [01:10<00:02, 4267.86it/s]

tf-idf method

In [45]:
corpusTables = []

for tables in formattedTables:
    
    corpusTables.append(tables[3])

In [8]:
corpusArticles = []

for articles in formattedArticles:
    
    corpusArticles.append(articles[1])

#temp1 = np.asarray(formattedArticles)
#corpusArticles = temp1[:,1]

In [46]:
vectorizer = TfidfVectorizer(stop_words='english', analyzer='word')
tablesByCharacteristics = vectorizer.fit_transform(corpusTables)

In [48]:
articlesByCharacteristics = vectorizer.transform(corpusArticles)

In [11]:
def getTotalTablesByArticle(idArticle):

    countTopTables = 0

    for tables in formattedTables:
    
        if tables[0] == int(idArticle):
        
             countTopTables = countTopTables + 1

    return countTopTables

In [12]:
def getIdRankedTalbes(topK,distanceVector):

    idRankedTables = []

    for topkDistance in topK:
    
        #index = distanceVector.index(topkDistance)
        
        index = np.where(distanceVector == topkDistance)
         
        indexColummun = index[0][0]
        
        idRankedTables.append(formattedTables[indexColummun][0])

    return idRankedTables

In [13]:
def getAccuracy(idRankedTables, idQueryGoal):

    totalRanquedTables = len(idRankedTables)

    match = 0

    for idTable in idRankedTables:
    
        if idTable == idQueryGoal:
    
            match = match + 1

    accuracy = match / totalRanquedTables
    
    return accuracy

In [49]:
accuracy = []

#articlesByCharacteristics.shape[0])

for j in tqdm(range(100)):

    distanceVector = pairwise_distances(articlesByCharacteristics[j], tablesByCharacteristics, metric='cosine')

    idQueryGoal = int(formattedArticles[j][0])

    countTopTables = getTotalTablesByArticle(idQueryGoal)
    
    topK = nsmallest(countTopTables, distanceVector[0])

    idRankedTables = getIdRankedTalbes(topK,distanceVector[0])
    
    accuracy.append(getAccuracy(idRankedTables,idQueryGoal))


100%|██████████| 100/100 [01:22<00:00,  1.21it/s]

In [58]:
np.mean(accuracy)

0.05333333333333334